In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import seaborn as sns
from functools import reduce

In [1]:
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # 지구의 반지름 (단위: km)
    
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    distance = R * c
    return distance

# 예시 지점의 위도와 경도
seller_lat = 37.12345
seller_lon = -122.67890
buyer_lat = 37.54321
buyer_lon = -122.98765

distance = haversine_distance(seller_lat, seller_lon, buyer_lat, buyer_lon)
print(f"두 지점 사이의 거리: {distance:.2f} km")


두 지점 사이의 거리: 54.07 km


In [4]:
customer = pd.read_csv('./olist_customers_dataset.csv')
geo = pd.read_csv('./olist_geolocation_dataset.csv')
order_item = pd.read_csv('./olist_order_items_dataset.csv')
order_payments = pd.read_csv('./olist_order_payments_dataset.csv')
order_reviews = pd.read_csv('./olist_order_reviews_dataset.csv')
orders = pd.read_csv('./olist_orders_dataset.csv')
products = pd.read_csv('./olist_products_dataset.csv')
seller = pd.read_csv('./olist_sellers_dataset.csv')
name_trans = pd.read_csv('./product_category_name_translation.csv')

In [13]:
customer_geo = geo.copy()
seller_geo = geo.copy()

In [14]:
new_column_names = ['customer_' + col for col in customer_geo.columns]
customer_geo.columns = new_column_names
customer_geo.rename(columns={"customer_geolocation_zip_code_prefix": "customer_zip_code_prefix"}, inplace=True)
customer_geo

,customer_zip_code_prefix,customer_geolocation_lat,customer_geolocation_lng,customer_geolocation_city,customer_geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [15]:
new_column_names = ['seller_' + col for col in seller_geo.columns]
seller_geo.columns = new_column_names
seller_geo.rename(columns={"seller_geolocation_zip_code_prefix": "seller_zip_code_prefix"}, inplace=True)
seller_geo

,seller_zip_code_prefix,seller_geolocation_lat,seller_geolocation_lng,seller_geolocation_city,seller_geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [5]:
df1 = reduce(lambda x,y: pd.merge(x,y, on='order_id', how='inner'), [order_item, orders, order_payments, order_reviews])
df2 = reduce(lambda x,y: pd.merge(x,y, on='customer_id', how='inner'), [df1, customer])
df3 = reduce(lambda x,y: pd.merge(x,y, on='product_id', how='inner'), [df2, products])
df4 = reduce(lambda x,y: pd.merge(x,y, on='seller_id', how='inner'), [df3, seller])
df5 = reduce(lambda x,y: pd.merge(x,y, on='product_category_name', how='inner'), [df4, name_trans])

In [17]:
df6 = reduce(lambda x,y: pd.merge(x,y, on='customer_zip_code_prefix', how='inner'), [df5, customer_geo])
df7 = reduce(lambda x,y: pd.merge(x,y, on='seller_zip_code_prefix', how='inner'), [df6, seller_geo])

: 

: 

In [6]:
df7.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value', 'customer_id',
       'order_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'review_id', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'seller_zip_code_prefix', 'seller_city', 'seller_state',
       'product_category_name_english'],
      dtype='object')